# MNIST Hand Written Digit Recognition

# Import Required Packages

In [ ]:
## Suppress useless warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
from pylab import *
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
tf.logging.set_verbosity(old_v)
%matplotlib inline

# Define the Hyper Parameters

In [ ]:
learning_rate = 0.1
display_step = 50
epochs = 500

# Input Data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Load data
X_train = mnist.train.images
Y_train = mnist.train.labels
X_test = mnist.test.images
Y_test = mnist.test.labels

print ("\nShape of Train Image Data: ", X_train.shape)
print ("Shape of Train Label Data: ", Y_train.shape)
print ("Shape of Test Image Data: ", X_test.shape)
print ("Shape of Test Label Data: ", Y_test.shape)

# Plot some images in MNIST

In [ ]:
fig, ax = plt.subplots(6, 6, figsize = (12, 12))
fig.suptitle('First 36 images in MNIST')
fig.tight_layout(pad = 0.3, rect = [0, 0, 0.9, 0.9])
for x, y in [(i, j) for i in range(6) for j in range(6)]:
    ax[x, y].imshow(X_train[x + y * 6].reshape((28, 28)), cmap = 'gray')
    title = str(np.where(Y_train[x + y * 6]==1)[0])
    ax[x, y].set_title(title)

In [ ]:


# Input pixels, flattened
x = tf.placeholder("float", [None, 784])
# Known labels
y_ = tf.placeholder("float", [None,10])

# Variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
# Define model
y = tf.nn.softmax(tf.matmul(x,W) + b)

# Climb on cross-entropy

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

# Just initialize
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init) # reset values to wrong
# Define accuracy
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Actually train
train_acc = np.zeros(epochs//10)
test_acc = np.zeros(epochs//10)

for i in range(epochs):
    # Record summary data, and the accuracy
    print ("Epoch: ", i, ", Loss: ", sess.run(cross_entropy, {x: X_train.reshape([-1,784]), y_: Y_train}))
    sess.run([train_step,cross_entropy] , {x: X_train.reshape([-1,784]), y_: Y_train})

    if i % 10 == 0:
        # Check accuracy on train set
        A = sess.run (accuracy, {x: X_train.reshape([-1,784]),y_: Y_train})
        train_acc[i//10] = A
        # And now the test set
        A = sess.run (accuracy, {x: X_test.reshape([-1,784]), y_: Y_test})
        #print (A)
        test_acc[i//10] = A

curr_W, curr_b, curr_cross_entropy = sess.run([W, b, cross_entropy], {x: X_train.reshape([-1,784]), y_: Y_train})
print("Training Done...!!")

In [ ]:
curr_W, curr_b, curr_cross_entropy = sess.run([W, b, cross_entropy], {x: X_train.reshape([-1,784]), y_: Y_train})

# Accuracy Plots

In [ ]:
# Plot the accuracy curves
plt.plot(train_acc,'bo')
plt.plot(test_acc,'rx')
# Notice that accuracy flattens out
print("Training Accuracy", train_acc[-1])
print("Testing Accuracy", test_acc[-1])

In [ ]:
# Look at a subplot of the weights for each font
f, plts = plt.subplots(10, sharex=True, figsize=(3,30))

for i in range(10):
    plts[i].pcolor(curr_W[:,i].reshape([28,28]), cmap=plt.cm.gray_r)